# Gallery Example: Hyper/Hyper/1 Reentrant Queue

This example demonstrates a HyperExp/HyperExp/1 queueing system with reentrant routing:
- **Arrivals**: Hyperexponential distribution (very high SCV)
- **Service**: Hyperexponential distribution (high SCV)
- **Servers**: 2 servers
- **Scheduling**: FCFS
- **Reentrant**: Jobs switch class and pass through the queue again

This model has high variability in both arrivals and service.

In [ ]:
// Kotlin notebook
import jline.*
import jline.lang.*
import jline.lang.nodes.*
import jline.lang.processes.*
import jline.lang.constant.*
import jline.solvers.ctmc.*
import jline.solvers.fluid.*
import jline.solvers.mva.*
GlobalConstants.setVerbose(VerboseLevel.STD)

In [ ]:
fun gallery_hyphyp1_reentrant(): Network {
    """Create Hyper/Hyper/1-Reentrant queueing model"""
    val model = Network("Hyper/Hyper/1-Reentrant")
// Block 1: nodes
    val source = Source(model, "Source")
    val queue = Queue(model, "Queue", SchedStrategy.FCFS)
    queue.setNumberOfServers(2)
    val sink = Sink(model, "Sink")
// Block 2: classes
    val oclass1 = OpenClass(model, "Class1")
    val oclass2 = OpenClass(model, "Class2")
// HyperExp arrival (SCV=64) for Class1, Class2 is disabled at source
    source.setArrival(oclass1, HyperExp.fit_mean_and_scv(1, 64))
    source.setArrival(oclass2, Disabled())
// HyperExp service (SCV=4) for Class1, Exponential for Class2
    queue.setService(oclass1, HyperExp.fit_mean_and_scv(0.5, 4))
    queue.setService(oclass2, Exp(3))
// Block 3: topology with class switching (reentrant)
    val P = model.initRoutingMatrix()
    P.addRoute(oclass1, source, queue, 1.0)
    P.addClassSwitch(oclass1, oclass2, queue, queue, 1.0)
    P.addRoute(oclass2, queue, sink, 1.0)
    model.link(P)
    
    return model
// Create the model
val model = gallery_hyphyp1_reentrant()

## About High Variability Systems

This model combines:
- **Very high arrival variability** (SCV = 64): Extremely bursty arrivals
- **High service variability** (SCV = 4): Highly variable service times
- **Multiple servers** (k = 2): Provides some capacity buffering

The combination of high variability in both arrival and service processes leads to:
- Very long tails in waiting time distribution
- Unpredictable queue behavior
- Periods of extreme congestion

The multiple servers help mitigate some of this variability.

In [ ]:
// Solve with multiple solvers
println("\n=== Solver Results ===")
// CTMC Solver
val solver_ctmc = CTMC(model, "cutoff", 15)
val avg_table_ctmc = solver_ctmc.avgTable
println("\nCTMC Solver:")
println(avg_table_ctmc)
// Fluid Solver
val solver_fluid = FLD(model)
val avg_table_fluid = solver_fluid.avgTable
println("\nFluid Solver:")
println(avg_table_fluid)